In [1]:
import numpy as np
from tqdm.notebook import trange
from tqdm.notebook import tqdm
from dask import delayed
import dask.array as da

np.get_printoptions()#["threshold"]
np.set_printoptions(edgeitems=10, linewidth=150)

np.random.seed(13)

## Mathematical Model

#### Sets
- C	Set of campaigns.
- U	Set of customers.
- H	Set of channels
- D	Set of planning days.
- I	Set of quota categories.
- P	Set of priority categories.


In [2]:
C = 10 # number of campaigns
U = 1000 # number of customers.
H = 3 # number of channels.
D = 7 # number of planning days.
I = 3 # number of quota categories.
P = 10 # number of priority categories.

#### Parameters

##### - eligibility
$$
e_{cu}\left\{\begin{array}\\
        1 & \mbox{if }  customer\ u\ is\ eligible\ for\ campaign\ c\\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [3]:
e_cu = np.random.choice(2,(C, U))
#e_cu = np.ones((C, U), dtype='int8')

##### - quota categories
$$
q_{ic}\left\{\begin{array}\\
        1 & \mbox{if }  campaign\ c\ is\ a\ i^{th} type\ quota\ category\ campaign\ \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [4]:
q_ic = np.random.choice(2, (I,C))
#q_ic = np.zeros((I,C), dtype='int8')
q_ic

array([[1, 0, 0, 1, 1, 1, 0, 0, 1, 1],
       [1, 0, 0, 1, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 1, 1, 0, 1, 1, 0]])

##### - priority categories
$$r_{cp}: Priority\ value\ of\ campaign\ c\ regarding\ priority\ type\ p\$$

In [5]:
r_p = np.random.choice(100, P)
#r_p = np.ones(P, dtype='int8')
rp_c = np.array([r_p[r] for r in np.random.choice(P, C)])
rp_c

array([41, 17, 22, 17, 33, 67, 33, 41, 17, 41])

##### - blokage
$$b: Communication\ limit\ per\ person\ for\ the\ whole\ period\$$

In [6]:
b = 4

##### - daily blokage
$$k: Communication\ limit\ per\ person\ at\ each\ day\$$

In [7]:
k = 2

##### - campaign blockage
$$l_c: Communication\ limit\ per\ person\ for\ campaign\ c\$$

In [8]:
l_c = np.random.choice([2,3,4],C)
l_c

array([2, 3, 4, 3, 2, 2, 4, 3, 3, 3])

##### - quota limitations daily/weekly
$$
m_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\
$$
$$
n_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\ each\ day\
$$

In [9]:
m_i = np.random.choice([10,15,18],I)
n_i = np.random.choice([2,3,4],I)
#m_i = np.ones((I), dtype='int8')*10
#n_i = np.ones((I), dtype='int8')*10
(m_i, n_i)

(array([10, 15, 10]), array([2, 2, 4]))

#### - capacity for channel
$$
t_{h,d}: Capacity\ for\ channel\ h\ at\ day\ d.\
$$

In [10]:
t_hd = np.random.choice([700], (H, D))

In [11]:
class Constraint:
    def __init__(self, fnc, fixed_indicies):
        self.fnc = fnc
        self.fixed_indicies = fixed_indicies
        
    def __variable_matched(self, indicies):
        for (k,v) in self.fixed_indicies.items():
            if(indicies[k]!=v):
                return False
        return True
    
    def get_func(self):
        return self.fnc
    
    def execute_considering(self, X, indicies):
        '''
        indicies: tuple of indicies indicating that would change from 0->1
        X: is the variables array
        indicies = (4, 100, 0, 1 )
        X_cuhd[indicies]
        '''
        if not self.__variable_matched(indicies):
            return np.array([True])
        else:
            params = tuple([X]) + tuple([(v)for (k,v) in sorted(self.fixed_indicies.items())])
            return self.fnc(*params)
        
        
class Model:
    def __init__(self, constraints):
        self.constraints = constraints
        
    def execute(self, X, indicies):
        for c in self.constraints:
            res = c.execute_considering(X, indicies)
            if not res.all():
                return False
        return True

# Model

#### Variables
$$
X_{cuhd}\left\{\begin{array}\\
        1 & \mbox{if } Campaign\ c\ will\ be\ sent\ to\ customer\ u\ through\ Channel\ h\ at\ Day\ d \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

## Maximize
$$\sum_{p \in P}\sum_{c \in C}\sum_{u \in U}\sum_{h \in H}\sum_{d \in D}\,X_{cuhd}\ r_{cp}$$

##### Binary variable (10)
$$
X_{cuhd} \in \{ 1,0 \},\hspace{35pt} \forall c \in C ,\forall u \in U,\forall d \in D, \forall h \in H
$$

In [12]:
X_cuhd = np.zeros((C,U,H,D), dtype='int')

## subject to

##### - eligibility (2)

$$
X_{cuhd}  \leq e_{cu},\hspace{35pt} \forall h \in H,\forall d \in D
$$

In [13]:
eligibility = lambda X, c, u, h, d: X[c,u,h,d]<=e_cu[c,u]

##### - use one channel (3)
$$
\sum_{h}X_{cuhd} \le 1,\hspace{35pt} \forall c \in C \, \forall u \in U,\forall d \in D
$$

In [14]:
one_channel = lambda X, c, u, d: X[c,u,:,d].sum() <= 1

##### - weekly communication limitation (4)
$$
\sum_{h \in H}\sum_{c \in C}\sum_{d \in D} X_{cuhd}\le b,\hspace{35pt} \forall u \in U
$$

In [15]:
weekly_limitation = lambda X, u: X[:,u,:,:].sum() <= b

##### - daily communication limitation (5)
$$
\sum_{h \in H}\sum_{c \in C} X_{cuhd}\le k,\hspace{35pt} \forall u \in U, \forall d \in D
$$

In [16]:
daily_limitation = lambda X, u, d: X[:,u,:,d].sum() <= k

##### - campaign communication limit(6)
$$
\sum_{d \in D}\sum_{h \in H} X_{cuhd}\le l_c,\hspace{35pt} \forall c \in C,\forall u \in U;
$$

In [17]:
campaign_limitation = lambda X, c, u: X[c,u,:,:].sum() <= l_c[c]

##### - weekly quota(7)
$$
\sum_{d \in D}\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le m_i,\hspace{35pt} \forall u \in U, \forall i \in I
$$

In [18]:
weekly_quota = lambda X, u: np.all(q_ic[:,c] * X[c,u,:,:].sum()<=m_i)

##### - daily quota(8)
$$
\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le n_i,\hspace{35pt} \forall u \in U,\, \forall d \in D, \forall i \in I
$$

In [19]:
daily_quota = lambda X, u, d: np.all(q_ic[:,c] * X[c,u,:,d].sum()<=n_i)

##### Channel capacity (9)
$$
\sum_{c \in C}\sum_{u \in U}{X_{cuhd}}\le t_{hd},\hspace{35pt} \forall d \in D,\, \forall h \in H
$$

In [20]:
channel_capacity = lambda X, h, d: X[:,:,h,d].sum() <= t_hd[h,d]

In [21]:
def check(X, c, u, h, d):
    if not eligibility(X_cuhd, c, u, h, d):
        return False
    if not channel_capacity(X_cuhd, h, d):
        return False
    if not weekly_limitation(X_cuhd, u):
        return False
    if not daily_limitation(X_cuhd, u, d):
        return False
    if not campaign_limitation(X_cuhd, c, u):
        return False
    if not one_channel(X_cuhd, c, u, d):
        return False
    if not daily_quota(X_cuhd, u, d):
        return False
    if not weekly_quota(X_cuhd, u):
        return False
    return True



In [22]:
%%time
#cons = [Constraint(eligibility, {0:c, 1:u, 2:h, 3:d}) for u in range(U) for c in range(C) for h in range(H) for d in range(D)]
#m=Model(cons)
for c in tqdm(np.argsort(-rp_c), desc="Campaigns Loop"):
    for h in range(H):#trange(H, desc=f"Channels Loop at Day-{d}, Campapaign-{c}"):
        for d in range(D):#trange(D, desc=f"Days Loop for campaign-{c}"):
            for u in range(U):#trange(U, desc=f"Users Loop On Campaign-{c}"):
                X_cuhd[c,u,h,d]=1
                if not check(X_cuhd, c, u, h, d):
                    X_cuhd[c,u,h,d]=0

np.matmul(rp_c, X_cuhd.sum(axis=(1,2,3)))

Campaigns Loop:   0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 5.15 s, sys: 45.4 ms, total: 5.19 s
Wall time: 5.22 s


181895

##### - check criterion

- Sort Campaigns by rp_c
- Map every criteria to function

##kucuk sayilar ile algoritmayi test edelim, ve bb solution burada validate edelim.